# RNN Model with crf

In [1]:
import rnn_dataset
import rnn_classifier
from crf import CRF
import torch
from data_utils import Vocabulary
from torch.utils.data import Dataset, DataLoader, random_split
from collections import Counter
import itertools
import torch.nn as nn
from tqdm import tqdm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from torch.nn.utils.rnn import pad_sequence
import pyconll


In [2]:
#define the hyperparameters
batch_size    = 16
window_size   =  6#left context and right context
lr            = 1e-3
device        = "cpu"
epochs        = 20
emb_size      = 64
hidden_size   = 64
nb_layers     = 2
drop_out      = 0.1

In [3]:
trainset     = rnn_dataset.MweRnnDataset("corpus/train.conllu",  isTrain = True)
testset      = rnn_dataset.MweRnnDataset("corpus/test.conllu")


_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
PREF
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
a
ET
fortiori
ET
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
for
ET
a
ET
Free
ET
Kuwait
ET
_
_
en
ET
The
ET
disinformation
ET
campaign
ET
The
ET
en
ET
Lessons
ET
in
ET
how
ET
to
ET
lie
ET
about
ET
The
ET
A

In [11]:
for x, d, tag in trainset.get_loader(shuffle = True):
    print(list(trainset.tags_vocab.rev_lookup(int(t)) for t in tag.squeeze(0)))
    break
    

['B_CL', 'B_AUX', 'B_V', 'B_P', 'B_D', 'B_N', 'B_PROPN', 'B_PONCT', 'B_P', 'B_D', 'B_N', 'B_P', 'B_PROPN', 'B_PONCT', '<pad>']


In [4]:
model = rnn_classifier.MweRNN(
    name         = "rnn",
    toks_vocab   = trainset.toks_vocab,
    tags_vocab   = trainset.tags_vocab, 
    deprel_vocab = trainset.deprel_vocab,
    emb_size     = emb_size, 
    hidden_size  = hidden_size, 
    drop_out     = 0.)

In [5]:
model.train_model(trainset,testset, epochs= epochs, lr=lr, batch_size = batch_size, split_train=0.8)

100%|█████████████████████████████████████████| 146/146 [00:02<00:00, 60.76it/s]


Epoch 0 | Mean train loss  46.4963 |  Mean dev loss  29.1421 



100%|█████████████████████████████████████████| 146/146 [00:02<00:00, 50.56it/s]


Epoch 1 | Mean train loss  24.7811 |  Mean dev loss  21.4905 



100%|█████████████████████████████████████████| 146/146 [00:02<00:00, 55.16it/s]


...

Epoch 18 | Mean train loss  2.1079 |  Mean dev loss  2.1175 



100%|█████████████████████████████████████████| 146/146 [00:02<00:00, 62.43it/s]


Epoch 19 | Mean train loss  1.9092 |  Mean dev loss  1.8602 



  0%|                                                   | 0/105 [00:00<?, ?it/s]


AttributeError: 'bool' object has no attribute 'sum'

In [ ]:
TP, FP, FN, average_precision, average_recall, average_f1_score, weighted_f1_score, weighted_recall, weighted_precision = model.evaluate()

In [7]:
num_tags = len(model.tags_vocab)
# print(num_tags)
TP = torch.zeros(num_tags)
FP = torch.zeros(num_tags)
FN = torch.zeros(num_tags)
class_counts = torch.zeros(num_tags)
with torch.no_grad():
            
    for X_toks, deprel, Y_golds in tqdm(testset.get_loader(batch_size = 500)):
    # Forward pass
        logprobs, masks = model.forward(X_toks)
        best_score, best_paths = model.crf(logprobs, masks) #viterbi
        #best_paths = pad_sequence(best_paths, padding_value= testset.tags_vocab["<pad>"]).T
        #print(best_paths.shape)
        # Mask out the padding positions
        #print(best_paths)
        
        for path, gold, x in zip(best_paths, Y_golds, X_toks):
            print(f"token {list(model.toks_vocab.rev_lookup(int(i)) for i in x if i!= 1)}")
            print(f"prediction {list(model.tags_vocab.rev_lookup(i) for i in path)}")
            print(f"gold {list(model.tags_vocab.rev_lookup(int(i)) for i in gold if i!= 1)}")
        print(Y_golds.shape)
        # Update confusion matrix
        for tag in range(num_tags):
            TP[tag] += ((best_paths == tag) & (Y_golds == tag)).sum()
            FP[tag] += ((best_paths == tag) & (Y_golds != tag)).sum()
            FN[tag] += ((best_paths != tag) & (Y_golds == tag)).sum()
            class_counts[tag] += (Y_golds == tag).sum()
            

  0%|                                                     | 0/4 [00:00<?, ?it/s]

token ['Gutenberg']
prediction ['B_ADV']
gold ['B_N']
token ['Cette', 'exposition', 'nous', 'apprend', 'que', 'dès', 'le', 'XIIe', 'siècle', ',', 'à', '<unk>', ',', 'entre', 'autres', 'sites', ',', 'une', 'industrie', '<unk>', 'existait', '.']
prediction ['B_DET', 'B_NOUN', 'B_PRON', 'B_VERB', 'B_ADV', 'B_ADP', 'B_DET', 'B_ADJ', 'B_NOUN', 'B_PUNCT', 'B_ADP', 'B_PROPN', 'B_PUNCT', 'B_ADP', 'B_ADJ', 'B_NOUN', 'B_PUNCT', 'B_DET', 'B_NOUN', 'B_AUX', 'B_VERB', 'B_PUNCT']
gold ['B_D', 'B_N', 'B_CL', 'B_V', 'B_C', 'B_P', 'B_D', 'B_A', 'B_N', 'B_PONCT', 'B_P', 'B_N', 'B_PONCT', 'B_P', 'B_A', 'B_N', 'B_PONCT', 'B_D', 'B_N', 'B_A', 'B_V', 'B_PONCT']
token ['à', 'peu', 'près', 'au', 'à', 'le', 'même', 'moment', 'que', 'Gutenberg', '<unk>', "l'", 'imprimerie', ',', '<unk>', '<unk>', 'créait', 'en', '1450', 'la', 'première', 'forge', 'à', '<unk>', ',', 'à', "l'", 'actuel', 'emplacement', 'du', 'de', 'le', '<unk>', '.']
prediction ['B_ADP', 'B_ADV', 'B_ADV', 'B__', 'B_ADP', 'B_DET', 'B_ADJ', 'B_NOUN

 25%|███████████▎                                 | 1/4 [00:01<00:03,  1.32s/it]

gold ['B_N', 'B_P', 'B_N', 'B_PONCT', 'B_D', 'B_N', 'B_A']
token ['Trois', 'personnes', 'ont', 'été', 'gravement', 'blessées', ',', 'dimanche', 'soir', 'vers', '17', 'h', '40', 'à', 'la', 'suite', "d'", 'un', 'accident', 'spectaculaire', 'qui', "s'", 'est', 'produit', 'à', "l'", 'entrée', 'du', 'de', 'le', 'village', 'de', '<unk>', 'sur', 'la', 'départementale', '32', '(', "L'", 'Est', '<unk>', 'du', 'de', 'le', '5', 'juillet', ')', '.']
prediction ['B_NUM', 'B_NOUN', 'B_AUX', 'B_AUX', 'B_ADV', 'B_VERB', 'B_PUNCT', 'B_NOUN', 'B_NOUN', 'B_ADP', 'B_NUM', 'B_NOUN', 'B_NUM', 'B_ADP', 'B_DET', 'B_NOUN', 'B_ADP', 'B_DET', 'B_NOUN', 'B_ADJ', 'B_PRON', 'B_PRON', 'B_AUX', 'B_VERB', 'B_ADP', 'B_DET', 'B_NOUN', 'B__', 'B_ADP', 'B_DET', 'B_NOUN', 'B_ADP', 'B_PROPN', 'B_ADP', 'B_DET', 'B_NOUN', 'B_NUM', 'B_PUNCT', 'B_PROPN', 'B_PROPN', 'B_PROPN', 'B__', 'B_ADP', 'B_DET', 'B_NUM', 'B_NOUN', 'B_PUNCT', 'B_PUNCT']
gold ['B_D', 'B_N', 'B_V', 'B_V', 'B_ADV', 'B_V', 'B_PONCT', 'B_N', 'B_N', 'B_P', 'B_D',

 50%|██████████████████████▌                      | 2/4 [00:02<00:02,  1.33s/it]

token ['-', 'Chez', 'les', 'patients', 'qui', 'bénéficient', "d'", 'une', 'intervention', '<unk>', 'destinée', 'à', 'traiter', 'des', '<unk>']
prediction ['B_PUNCT', 'B_ADP', 'B_DET', 'B_NOUN', 'B_PRON', 'B_VERB', 'B_ADP', 'B_DET', 'B_NOUN', 'B_AUX', 'B_VERB', 'B_ADP', 'B_VERB', 'B_DET', 'B_PROPN']
gold ['B_PONCT', 'B_P', 'B_D', 'B_N', 'B_PRO', 'B_V', 'B_P', 'B_D', 'B_N', 'B_A', 'B_V', 'B_P', 'B_V', 'B_D', 'B_N']
token ['<unk>', 'SONT', 'LES', '<unk>', 'À', '<unk>', '<unk>', "D'", '<unk>', '<unk>']
prediction ['B_AUX', 'B_VERB', 'B_PROPN', 'B_PROPN', 'B_PROPN', 'B_PROPN', 'B_PROPN', 'B_PROPN', 'B_PROPN', 'B_PROPN']
gold ['B_A', 'B_V', 'B_D', 'B_N', 'B_P', 'B_V', 'B_P', 'I_P', 'B_V', 'B_N']
token ["N'", 'utilisez', 'jamais', '<unk>', ':']
prediction ['B_PART', 'B_VERB', 'B_ADV', 'B_PROPN', 'B_PUNCT']
gold ['B_ADV', 'B_V', 'B_ADV', 'B_N', 'B_PONCT']
token ['-', 'si', 'vous', 'êtes', '<unk>', '(', 'allergique', ')', 'à', 'la', '<unk>', 'ou', 'à', "l'", 'un', 'des', 'de', 'les', 'autres', 

 75%|█████████████████████████████████▊           | 3/4 [00:04<00:01,  1.37s/it]

token ['Avant', "l'", 'administration', ',', 'il', 'faut', 'laisser', 'la', 'solution', '<unk>', 'atteindre', 'la', 'température', 'ambiante', '.']
prediction ['B_P', 'B_D', 'B_N', 'B_PONCT', 'B_CL', 'B_V', 'B_V', 'B_D', 'B_N', 'B_V', 'B_V', 'B_D', 'B_N', 'B_A', 'B_PONCT']
gold ['B_P', 'B_D', 'B_N', 'B_PONCT', 'B_CL', 'B_V', 'B_V', 'B_D', 'B_N', 'B_V', 'B_V', 'B_D', 'B_N', 'B_A', 'B_PONCT']
token ['Que', 'contient', '<unk>']
prediction ['B_PRO', 'B_V', 'B_V']
gold ['B_PRO', 'B_V', 'B_N']
token ['-', 'La', 'substance', 'active', 'est', "l'", 'acide', '<unk>', '.']
prediction ['B_PUNCT', 'B_DET', 'B_NOUN', 'B_ADJ', 'B_VERB', 'B_DET', 'B_NOUN', 'B_VERB', 'B_PUNCT']
gold ['B_PONCT', 'B_D', 'B_N', 'B_A', 'B_V', 'B_D', 'B_N', 'B_A', 'B_PONCT']
token ['Chaque', '<unk>', 'de', '100', 'ml', 'de', 'solution', 'contient', '5', 'mg', "d'", 'acide', '<unk>', '<unk>', ',', 'ce', 'qui', '<unk>', 'à', '<unk>', 'mg', "d'", 'acide', '<unk>', '<unk>', '.']
prediction ['B_DET', 'B_PROPN', 'B_ADP', 'B_NUM'

100%|█████████████████████████████████████████████| 4/4 [00:04<00:00,  1.12s/it]

token ['Il', "n'", 'y', 'a', 'aucun', 'motif', 'pour', "qu'", 'ils', 'y', 'soient', '.']
prediction ['B_PRON', 'B_PART', 'B_CL', 'B_V', 'B_D', 'B_N', 'B_P', 'I_C', 'B_CL', 'B_ADV', 'B_VERB', 'B_PUNCT']
gold ['B_CL', 'B_ADV', 'B_CL', 'B_V', 'B_D', 'B_N', 'B_P', 'I_C', 'B_CL', 'B_CL', 'B_V', 'B_PONCT']
token ['Ils', "n'", 'ont', 'été', 'impliqués', 'dans', 'aucune', 'action', 'terroriste', 'ou', 'militaire', '.']
prediction ['B_PRON', 'B_PART', 'B_AUX', 'B_AUX', 'B_VERB', 'B_ADP', 'B_DET', 'B_NOUN', 'B_ADJ', 'B_CONJ', 'B_ADJ', 'B_PUNCT']
gold ['B_CL', 'B_ADV', 'B_V', 'B_V', 'B_V', 'B_P', 'B_D', 'B_N', 'B_A', 'B_C', 'B_A', 'B_PONCT']
token ['Je', 'crois', 'que', 'nous', 'devrions', 'nous', 'pencher', 'sur', 'ce', 'problème', '.']
prediction ['B_CL', 'B_V', 'B_C', 'B_CL', 'B_V', 'B_CL', 'B_V', 'B_P', 'B_D', 'B_N', 'B_PONCT']
gold ['B_CL', 'B_V', 'B_C', 'B_CL', 'B_V', 'B_CL', 'B_V', 'B_P', 'B_D', 'B_N', 'B_PONCT']
token ['<unk>', '<unk>', ',', 'la', 'docteur', '<unk>', 'condamnée', 'le', '1

In [32]:
TP

tensor([0.0000e+00, 1.6430e+03, 0.0000e+00, 1.4342e+05, 2.0000e+01, 0.0000e+00,
        0.0000e+00, 5.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.1000e+01,
        3.5000e+01, 3.0000e+00, 0.0000e+00, 0.0000e+00, 7.8000e+01, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00])